# Pivoting “long” to “wide” Format

A common way to store multiple time series in databases and CSV is in so-called long or stacked format:

In [35]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [46]:
data = pd.read_csv('../../CSV Files/FAO_data.csv')

data.head(5)

,Item Code,Year,Value,Item
0,3102,1961,41659,nitrogen N (total)
1,3103,1961,500,phosphate P2O5 (total)
2,3103,1962,210,phosphate P2O5 (total)
3,3102,1962,41160,nitrogen N (total)
4,3102,1963,67620,nitrogen N (total)


Data is frequently stored this way in relational databases like MySQL as a fixed schema (column names and data types) allows the number of distinct values in the item column to increase or decrease as data is added or deleted in the table. In the above example date and item would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins and programmatic queries in many cases. The downside, of course, is that the data may not be easy to work with in long format; you might prefer to have a DataFrame containing one column per distinct item value indexed by timestamps in the date column. DataFrame’s pivot method performs exactly this transformation:

In [50]:
pivoted = data.pivot('Year', 'Item', 'Value')

pivoted.head()

Item,nitrogen N (total),phosphate P2O5 (total),potash K2O (total)
Year,,,
1961,41659.0,500.0,NaN
1962,41160.0,210.0,NaN
1963,67620.0,630.0,NaN
1964,84147.0,1029.0,NaN
1965,69242.0,1245.0,NaN


The first two values passed are the columns to be used as the row and column index, and finally an optional value column to fill the DataFrame. Suppose you had two value columns that you wanted to reshape simultaneously:

In [51]:
data['value2'] = np.random.randn(len(data))

In [54]:
data.tail(5)

,Item Code,Year,Value,Item,value2
167,3102,2018,3446922,nitrogen N (total),1.354432
168,3104,2018,54173,potash K2O (total),-0.354231
169,3104,2019,47260,potash K2O (total),0.516736
170,3102,2019,3505356,nitrogen N (total),-0.199051
171,3103,2019,1099707,phosphate P2O5 (total),-2.845489


By omitting the last argument, you obtain a DataFrame with hierarchical columns:

In [61]:
pivoted = data.pivot('Year', 'Item')

pivoted['Value'][-5:]

Item,nitrogen N (total),phosphate P2O5 (total),potash K2O (total)
Year,,,
2015,3160507.0,993338.0,25291.0
2016,3241907.0,1209145.0,28954.0
2017,3446922.0,1257773.0,54173.0
2018,3446922.0,1257773.0,54173.0
2019,3505356.0,1099707.0,47260.0


Note that pivot is just a shortcut for creating a hierarchical index using set_index and reshaping with unstack:

In [60]:
unstacked = data.set_index(['Year', 'Item']).unstack('Item')

unstacked[-5:]

Item Code                                              \
Item  nitrogen N (total)  phosphate P2O5 (total)  potash K2O (total)   
Year                                                                   
2015              3102.0                  3103.0              3104.0   
2016              3102.0                  3103.0              3104.0   
2017              3102.0                  3103.0              3104.0   
2018              3102.0                  3103.0              3104.0   
2019              3102.0                  3103.0              3104.0   

                   Value                                              \
Item  nitrogen N (total)  phosphate P2O5 (total)  potash K2O (total)   
Year                                                                   
2015           3160507.0                993338.0             25291.0   
2016           3241907.0               1209145.0             28954.0   
2017           3446922.0               1257773.0             54173.0   
2018           3446922.0               1257773.0             54173.0   
2019           3505356.0               1099707.0             47260.0   

                  value2                                              
Item  nitrogen N (total)  phosphate P2O5 (total)  potash K2O (total)  
Year                                                                  
2015           -0.070213               -0.837996           -0.316840  
2016           -1.787382               -0.118625           -0.571296  
2017           -0.945466                1.687901           -0.968440  
2018            1.354432                0.189847           -0.354231  
2019           -0.199051               -2.845489            0.516736